In [2]:
import pandas as pd
import numpy as np
import os

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [8]:
Tcredits = pd.read_csv("../datasets/TMDB/tmdb_5000_credits.csv")
Tmovies = pd.read_csv("../datasets/TMDB/tmdb_5000_movies.csv")
Tratings = pd.read_csv("../datasets/The_Movies/ratings_small.csv")
Tlinks = pd.read_csv("../datasets/The_Movies/links_small.csv")

In [24]:
#Tmeta = pd.read_csv("../datasets/The_Movies/movies_metadata.csv")
Tmeta = pd.read_csv("../datasets/The_Movies/movies_metadata.csv")
print("metadata file is corrupted but same date is present in tmdb_5000_movies.csv datafile")
pd.DataFrame({'feature':Tmeta.dtypes.index, 'dtype':Tmeta.dtypes.values})
# Tmeta = pd.read_csv(os.path.join(PATH,"movies_metadata.csv"), low_memory=False)
# Tmeta[Tmeta['popularity'] == 'Beware Of Frost Bites']
# loc = Tmeta.index[Tmeta['popularity'] == 'Beware Of Frost Bites'].tolist()
# Tmeta.iloc[loc]
# Tmeta = Tmeta.drop(Tmeta.index[35587])
# Tmeta.astype({'popularity':float})

metadata file is corrupted but same date is present in tmdb_5000_movies.csv datafile


/home/ben/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


feature    dtype
0                   adult   object
1   belongs_to_collection   object
2                  budget   object
3                  genres   object
4                homepage   object
5                      id   object
6                 imdb_id   object
7       original_language   object
8          original_title   object
9                overview   object
10             popularity   object
11            poster_path   object
12   production_companies   object
13   production_countries   object
14           release_date   object
15                revenue  float64
16                runtime  float64
17       spoken_languages   object
18                 status   object
19                tagline   object
20                  title   object
21                  video   object
22           vote_average  float64
23             vote_count  float64

In [26]:
#Tcredits = pd.read_csv("../datasets/The_Movies/credits.csv")
#Credits can be fetched from TMDB dataset
pd.DataFrame({'feature':Tmovies.dtypes.index, 'dtype':Tmovies.dtypes.values})

feature    dtype
0                 budget    int64
1                 genres   object
2               homepage   object
3                     id    int64
4               keywords   object
5      original_language   object
6         original_title   object
7               overview   object
8             popularity  float64
9   production_companies   object
10  production_countries   object
11          release_date   object
12               revenue    int64
13               runtime  float64
14      spoken_languages   object
15                status   object
16               tagline   object
17                 title   object
18          vote_average  float64
19            vote_count    int64

In [10]:
#print(Tmovies.columns)
Tmovies.head(2)

budget                                             genres  \
0  237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   

                                       homepage     id  \
0                   http://www.avatarmovie.com/  19995   
1  http://disney.go.com/disneypictures/pirates/    285   

                                            keywords original_language  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...                en   
1  [{"id": 270, "name": "ocean"}, {"id": 726, "na...                en   

                             original_title  \
0                                    Avatar   
1  Pirates of the Caribbean: At World's End   

                                            overview  popularity  \
0  In the 22nd century, a paraplegic Marine is di...  150.437577   
1  Captain Barbossa, long believed to be dead, ha...  139.082615   

                                production_companies  \
0  [{"name": "Ingenious Film Partners", "id": 289...   
1  [{"name": "Walt Disney Pictures", "id": 2}, {"...   

                                production_countries release_date     revenue  \
0  [{"iso_3166_1": "US", "name": "United States o...   2009-12-10  2787965087   
1  [{"iso_3166_1": "US", "name": "United States o...   2007-05-19   961000000   

   runtime                                   spoken_languages    status  \
0    162.0  [{"iso_639_1": "en", "name": "English"}, {"iso...  Released   
1    169.0           [{"iso_639_1": "en", "name": "English"}]  Released   

                                          tagline  \
0                     Enter the World of Pandora.   
1  At the end of the world, the adventure begins.   

                                      title  vote_average  vote_count  
0                                    Avatar           7.2       11800  
1  Pirates of the Caribbean: At World's End           6.9        4500

In [11]:
Tmovies_1 = Tmovies[['genres','id','keywords','title','overview','popularity','production_companies','release_date','vote_average','vote_count']]
Tmovies_1.head(2)

genres     id  \
0  [{"id": 28, "name": "Action"}, {"id": 12, "nam...  19995   
1  [{"id": 12, "name": "Adventure"}, {"id": 14, "...    285   

                                            keywords  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...   
1  [{"id": 270, "name": "ocean"}, {"id": 726, "na...   

                                      title  \
0                                    Avatar   
1  Pirates of the Caribbean: At World's End   

                                            overview  popularity  \
0  In the 22nd century, a paraplegic Marine is di...  150.437577   
1  Captain Barbossa, long believed to be dead, ha...  139.082615   

                                production_companies release_date  \
0  [{"name": "Ingenious Film Partners", "id": 289...   2009-12-10   
1  [{"name": "Walt Disney Pictures", "id": 2}, {"...   2007-05-19   

   vote_average  vote_count  
0           7.2       11800  
1           6.9        4500

In [12]:
# the id column names are different in both datasets. 
Tmovies_2 = Tmovies_1.rename(columns={"id":"movie_id"})
Tcredits_2 = Tcredits.drop(columns=["title"])

In [13]:
#combine to single dataframe
print("Duplicate movie id's conunt in movies df: ",Tmovies_2.movie_id.duplicated().sum())
print("Duplicate movie id's conunt in credits df: ",Tcredits_2.movie_id.duplicated().sum())
Tmovies_3 = Tmovies_2.merge(Tcredits_2,on="movie_id")
Tmovies_3.head(2)

Duplicate movie id's conunt in movies df:  0
Duplicate movie id's conunt in credits df:  0


genres  movie_id  \
0  [{"id": 28, "name": "Action"}, {"id": 12, "nam...     19995   
1  [{"id": 12, "name": "Adventure"}, {"id": 14, "...       285   

                                            keywords  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...   
1  [{"id": 270, "name": "ocean"}, {"id": 726, "na...   

                                      title  \
0                                    Avatar   
1  Pirates of the Caribbean: At World's End   

                                            overview  popularity  \
0  In the 22nd century, a paraplegic Marine is di...  150.437577   
1  Captain Barbossa, long believed to be dead, ha...  139.082615   

                                production_companies release_date  \
0  [{"name": "Ingenious Film Partners", "id": 289...   2009-12-10   
1  [{"name": "Walt Disney Pictures", "id": 2}, {"...   2007-05-19   

   vote_average  vote_count  \
0           7.2       11800   
1           6.9        4500   

                                                cast  \
0  [{"cast_id": 242, "character": "Jake Sully", "...   
1  [{"cast_id": 4, "character": "Captain Jack Spa...   

                                                crew  
0  [{"credit_id": "52fe48009251416c750aca23", "de...  
1  [{"credit_id": "52fe4232c3a36847f800b579", "de...

In [14]:
Tratings_3 = Tratings.rename(columns={"movieId":"movie_id"})
Tratings_3.head(2)

userId  movie_id  rating   timestamp
0       1        31     2.5  1260759144
1       1      1029     3.0  1260759179

In [15]:
Tmovies_3.isnull().sum()

genres                  0
movie_id                0
keywords                0
title                   0
overview                3
popularity              0
production_companies    0
release_date            1
vote_average            0
vote_count              0
cast                    0
crew                    0
dtype: int64

In [16]:
Tmovies_3 = Tmovies_3.fillna("")
Tmovies_3.isnull().sum()

genres                  0
movie_id                0
keywords                0
title                   0
overview                0
popularity              0
production_companies    0
release_date            0
vote_average            0
vote_count              0
cast                    0
crew                    0
dtype: int64

In [126]:
print("duplicated records", Tmovies_1.duplicated().sum())

duplicated records 0


In [17]:
Tmovies_3['overview'].head()

0    In the 22nd century, a paraplegic Marine is di...
1    Captain Barbossa, long believed to be dead, ha...
2    A cryptic message from Bond’s past sends him o...
3    Following the death of District Attorney Harve...
4    John Carter is a war-weary, former military ca...
Name: overview, dtype: object

In [19]:
vectorizer = TfidfVectorizer(stop_words='english')
vectorized = vectorizer.fit_transform(Tmovies_3['overview'])
print("shape" , vectorized.shape)
# it is mxn matrix with m equal to the total rows of  dataset. 
# n is total no of different words used in overview column
# now if given a row or title we need to find other rows with similar data in the matrix

shape (4803, 20978)


In [20]:
similarityMatrix = linear_kernel(vectorized, vectorized)
# it finds the similarity with all other rows in matrix and returns a list 
similarityMatrix

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.02160533, 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.01488159, 0.        ,
        0.        ],
       ...,
       [0.        , 0.02160533, 0.01488159, ..., 1.        , 0.01609091,
        0.00701914],
       [0.        , 0.        , 0.        , ..., 0.01609091, 1.        ,
        0.01171696],
       [0.        , 0.        , 0.        , ..., 0.00701914, 0.01171696,
        1.        ]])

Similar score matrix data can be fetched by using index of the required row. So if user searches for a particular movie, we need to get the index of that movie and then get the similartiy scores data. For this conversion instead of converting each time during search we can create a reverse map to store tiles as key & index as values. 

In [130]:
indexMap = pd.Series(Tmovies_3.index, index=Tmovies_3['title'])
indexMap.head()
#indexMap.duplicated().sum()

title
Avatar                                      0
Pirates of the Caribbean: At World's End    1
Spectre                                     2
The Dark Knight Rises                       3
John Carter                                 4
dtype: int64

Once we get similarity scores for a particular movie, we need to sort them to find top 10 similar movies indices & then fetch the titles to recommend to the user. As any movie will have 1 as similarity score when compared with itself, its own index appears at top of list. So we ignore it and take next 10 items from the results. 

In [134]:
#sorting
sim_sorted = np.argsort(similarityMatrix[0])
#top 10 
sim_indices = sim_sorted[::-1][:11]
#titles
sim_titles = Tmovies_3.iloc[sim_indices]['title']
sim_titles[1:]

3604                       Apollo 18
2130                    The American
634                       The Matrix
1341            The Inhabited Island
529                 Tears of the Sun
1610                           Hanna
311     The Adventures of Pluto Nash
847                         Semi-Pro
775                        Supernova
2628             Blood and Chocolate
Name: title, dtype: object

In [135]:
def overviewRecommender(name):
    index = indexMap[name]
    scores = similarityMatrix[index]
    print("Top 10 similarity scores", np.sort(scores)[::-1][1:11])
    sim_sorted = np.argsort(scores)
    top_10 = sim_sorted[::-1][1:11]
    print(top_10)
    recommendations = Tmovies_3.iloc[top_10]['title']
    return recommendations

In [136]:
overviewRecommender('Spider-Man')

Top 10 similarity scores [0.28269378 0.24281586 0.223084   0.20914974 0.18502292 0.16479718
 0.14647248 0.13076654 0.11323179 0.11282744]
[   5   38   20   30 1534  953 1720 2740 3216 1572]


5                    Spider-Man 3
38       The Amazing Spider-Man 2
20         The Amazing Spider-Man
30                   Spider-Man 2
1534                Arachnophobia
953     Gremlins 2: The New Batch
1720                     Kick-Ass
2740                  The New Guy
3216                     Election
1572    Forgetting Sarah Marshall
Name: title, dtype: object